In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastparquet

In [ ]:
df = pd.read_parquet("meow_df.parquet", engine = 'fastparquet')

df['time_diff'] = df.seconds_in_bucket - df.seconds_in_bucket_before

df = df.query("""time_id == time_id_before and stock_id == stock_id_before""")

df = df.drop(columns = ['time_id_before', 'seconds_in_bucket_before', 'stock_id_before', 'level_0'])

df['trade'] = (df.price >= df.ask_price1_before).astype(int) - (df.price <= df.bid_price1_before).astype(int)

In [ ]:
df['mid'] = (df.ask_price1 + df.bid_price1) / 2

df['spread'] = (df.ask_price1 - df.bid_price1)*10000 / df.mid

In [ ]:
np.log(df.time_diff.value_counts()).plot.hist(bins = 50)

In [ ]:
#meow moew

In [ ]:
target = pd.read_csv("baseline.csv",index_col="Unnamed: 0")



gb = df.groupby(by=['stock_id', 'time_id']).agg({"size":"sum", "spread": "mean","time_diff":"max"})
gb['log_time_diff'] = np.log(gb.time_diff)

In [ ]:
target = target.merge(gb.reset_index())

In [ ]:
avg_per_stock = target.groupby('stock_id')['log_time_diff'].transform('mean')

In [ ]:
target['timefunc'] = np.abs(target['log_time_diff']- avg_per_stock)**.5


#timef = gb.apply(func = timefunc, axis=1)

In [ ]:
target.timefunc.describe()

In [ ]:
np.log(df.spread).plot.hist(bins = 50)

In [ ]:
pd.Series(np.log(df['size'])).plot.hist(bins = 50)

In [ ]:
target.plot.scatter(x="spread", y="target",c= "stock_id")
#spread very high corr with vol. unsurprising

In [ ]:

pd.plotting.scatter_matrix(target.head(1000).drop(columns=['stock_id','time_id']))

#Size low corr with sigma: prob due to low corr size ~ volume

In [ ]:
target.plot.scatter(x= "timefunc", y = "target", c="stock_id")

#Hmm very liquid mala sigma, illiquid tez, mid liquid duze zmiany?

In [ ]:
target.corr().target.sort_values()
#cool

In [ ]:
target.to_csv("baseline2.csv")
df.to_parquet("clean_trades.parquet", engine='fastparquet')

df.columns

In [ ]:
target.head()

In [ ]:
target['stock_id'] = target['stock_id'].astype('category')

In [ ]:
from sklearn.metrics import r2_score

def rmsre(y_true, y_pred):
    return np.mean(((y_true - y_pred) / y_true)**2)**0.5

def print_stats(x, y, model):
    y_pred = model.predict(x)
    error = rmsre(y, y_pred)
    r2 = r2_score(y, y_pred)
    print(error)
    print(r2)

In [ ]:
from sklearn.model_selection import train_test_split

X = target.drop(columns=['time_id', 'log_time_diff', 'target'])
y = target['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y

In [ ]:
import xgboost as xgb

model_xgb = xgb.XGBRegressor(n_estimators=3000, max_depth=3, enable_categorical=True)
model_xgb.fit(X_train, y_train)
print_stats(X_test, y_test, model_xgb)

In [ ]:
print_stats(X_train, y_train, model_xgb)

In [ ]:
print(model_xgb.feature_importances_)

In [ ]:
X_train.columns

In [ ]:
len(X_train)

In [ ]:
target.stock_id.dtype

In [ ]:
target_1 = target[target['stock_id'] == 1]

In [ ]:
target_1.head()

In [ ]:
x = target_1.drop(columns=['stock_id', 'time_id', 'target'])
y = target_1['target']

x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=.8, test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
error = rmsre(y_test, y_pred)
print(error)
r2 = r2_score(y_test, y_pred)
print(r2)

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=7500, max_depth=3)
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)
error = rmsre(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(error)
print(r2)

In [ ]:
def print_stats(x, y, model):
    y_pred = model.predict(x)
    error = rmsre(y, y_pred)
    r2 = r2_score(y, y_pred)
    print(error)
    print(r2)

In [ ]:
print_stats(x_train, y_train, xgb_model)

In [ ]:
print_stats(x_test, y_test, xgb_model)

In [ ]:
target['target'].describe()

In [ ]:
quantile_99 = np.quantile(target['target'], q=0.99)
print(quantile_99)

In [ ]:
target_high_vol = target.query(f'target > {quantile_99}')

In [ ]:
print(len(target_high_vol) / len(target))

In [ ]:
target_high_vol.plot.scatter(x='target', y='sigma')